In [1]:
#Import Dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver

In [2]:
#Determine Path
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
#Select Executable Path
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser('chrome', **executable_path, headless=False)


# NASA Mars News

In [4]:
#Use Splinter to visit NASA News re Mars
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [5]:
#Create HTML Object

news_html = browser.html
news_soup = bs(news_html, "html.parser")

#Scrape title and paragraph
news_tit = news_soup.find_all('div', class_='content_title')[1].text
news_para = news_soup.find_all('div', class_='article_teaser_body')[0].text


In [6]:
#Print Result
print(f"""
TITLE: {news_tit}

PARAGRAPH: {news_para}
""")


TITLE: MOXIE Could Help Future Rockets Launch Off Mars

PARAGRAPH: NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.



# JPL Mars Space Images

In [7]:
#Use Splinter to visit NASA JPL Mars Space Images
jpl_url = "https://www.jpl.nasa.gov"
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)


In [8]:
#Create HTML Object
img_html = browser.html
img_soup = bs(img_html, "html.parser")



In [9]:
#Create Featured Image URL
img_url  = img_soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
featured_image_url = jpl_url + img_url



In [10]:
#Print Result
print(f"""Featured Image URL:

{featured_image_url}
""")

Featured Image URL:

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17564-1920x1200.jpg



# Mars Facts

In [11]:
#Use Splinter to visit Space Facts Page for Mars
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

In [12]:
#Read Object into pandas
mars_facts = pd.read_html(facts_url)
mars_df = mars_facts[0]
mars_df.columns = ['Parameter','Mars']

mars_df.head(10)

,Parameter,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
#Set Description as Index
mars_df.set_index('Parameter', inplace=True)


In [14]:
#Convert Table to HTML
facts_table = mars_df.to_html()
print(facts_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
    </tr>
    <tr>
      <th>Parameter</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemipheres

In [23]:
#Use Splinter to visit astrogeology website
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)

#Create HTML Object
hemi_html = browser.html
hemi_soup = bs(hemi_html, 'html.parser')

In [24]:
#set variables and retrieve information for us in loop
astro_url = 'https://astrogeology.usgs.gov'
hemi_results = hemi_soup.find('div', class_ = "result-list" )
hemispheres = hemi_results.find_all('div', class_ = 'item')
hemisphere_urls = []

#For loop to iterate through each image
for hemisphere in hemispheres:
    news_title = hemisphere.find('h3').text
    news_title = news_title.replace("Enhanced", "")
    link_end = hemisphere.find("a")["href"]
    image_link = astro_url + link_end    
    browser.visit(image_link)
    hemi_html = browser.html
    hemi_soup = bs(hemi_html, "html.parser")
    downloads = hemi_soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_urls.append({"title": news_title, "img_url": image_url})

In [25]:
print (hemisphere_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [26]:
#Store Data In Dictionary
mars_info = {
        "news_tit": news_tit,
        "news_para": news_para,
        "featured_image_url": featured_image_url,
        "mars_facts": facts_table,
        "hemisphere_urls": hemisphere_urls
}

mars_info

{'news_tit': 'MOXIE Could Help Future Rockets Launch Off Mars',
 'news_para': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17564-1920x1200.jpg',
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Parameter</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38